In [ ]:
import cv2
from utils import get_parking_spots_bboxes, empty_or_not
import numpy as np
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer

from google.colab.patches import cv2_imshow

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
def summarize_parking_status(occupied, available, timestamp):
    prompt = f"""
        Write a detailed but compact Markdown report for a parking lot monitoring system.

        Data:
        - Timestamp: {timestamp}
        - Occupied Spaces: {occupied}
        - Available Spaces: {available}

        Follow this EXACT Markdown template.
        DO NOT add or remove sections.
        Use bullet points under EVERY heading.
        Do NOT write plain sentences under headings.

        ### 📍 Snapshot
        - Total capacity: occupied + available (show numbers)
        - Occupancy rate: percentage with 1 decimal
        - Status: Comfortable (<70%), Busy (70–85%), Near capacity (>85%)

        ### 📈 Operational Insight
        - Exactly 2 bullet points
        - Each bullet <= 18 words

        ### ⚠️ Risks
        - Exactly 3 bullet points
        - Each bullet <= 14 words

        ### ✅ Recommendations
        - Exactly 3 bullet points
        - Each bullet <= 14 words

        ### 🧠 What to monitor next
        - Exactly 2 bullet points
        - Each bullet <= 12 words

        Rules:
        - No tables
        - No LaTeX or math formatting
        - No invented context
        - Total output <= 220 words
        """


    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=450
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    content = tokenizer.decode(output_ids, skip_special_tokens=True)

    return content

# Function to save data to Markdown
def save_report_to_md(report_data, filename="Parking_Report.md"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write("# 🚗 Parking Lot Analysis Report\n\n")
        f.write(f"**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        f.write("## 📊 Occupancy Summary Table\n\n")
        f.write("| Timestamp           | Occupied Spaces | Available Spaces |\n")
        f.write("|--------------------|-----------------|-----------------|\n")
        for entry in report_data:
            f.write(f"| {entry['timestamp']} | {entry['occupied']} | {entry['available']} |\n")

        f.write("\n## 📄 Detailed Analysis by LLM\n\n")
        for entry in report_data:
            f.write(f"### ⏰ Report at {entry['timestamp']}\n")
            f.write(f"- **Occupied Spaces:** {entry['occupied']}\n")
            f.write(f"- **Available Spaces:** {entry['available']}\n\n")

            # write as normal markdown
            f.write(entry["summary"].strip() + "\n\n")

            f.write("---\n\n")



In [ ]:
def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))

In [ ]:
mask = "data/mask_1920_1080.png"

video_path = "data/parking_1920_1080.mp4"

mask = cv2.imread(mask, 0)
connected_components = cv2.connectedComponentsWithStats(mask, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_components)

cap = cv2.VideoCapture(video_path)
window_name = 'frame'

spots_status = [None for j in spots]
frame_num = 0

window_closed = False  # Flag to track if the window was closed manually
step = 30
summary_interval = 300  # Generate summary every 300 frames

diffs = [None for j in spots]
prev_frame = None

# Initialize report data storage
report_data = []



In [ ]:
while cap.isOpened():
    ret, frame = cap.read()

    if frame_num % step == 0 and prev_frame is not None:
        for spot_index, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]

            diffs[spot_index] = calc_diff(spot_crop, prev_frame[y1:y1+h, x1:x1+w, :])

    #Once every 30 frames
    if frame_num % step == 0:
        if prev_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]

        for spot_index in arr_:
            spot = spots[spot_index]
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w, :]

            spot_status = empty_or_not(spot_crop)
            spots_status[spot_index] = spot_status

    if frame_num % step == 0:
        prev_frame = frame.copy()


    for spot_index, spot in enumerate(spots):
        spot_status = spots_status[spot_index]
        x1, y1, w, h = spots[spot_index]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1+h ), (0,255,0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1+h ), (0,0,255), 2)


    cv2.rectangle(frame, (80,20), (550,80), (0,0,0), -1)
    cv2.putText(frame, 'Available Spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    available_count = sum(spots_status)
    occupied_count = len(spots_status) - available_count
    # cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    # cv2.imshow(window_name, frame)


    if not ret:
        print("End of video or can't retrieve frame.")
        break

    # Check if 'q' is pressed or window is closed
    # if cv2.waitKey(1) == ord('q'):
    #     print("Exiting on 'q' press.")
    #     break

    # if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
    #     print("Window was closed manually.")
    #     window_closed = True
    #     break

    # Generate LLM Summary at intervals
    if frame_num % summary_interval == 0:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        llm_summary = summarize_parking_status(occupied_count, available_count, timestamp)
        report_data.append({
            "timestamp": timestamp,
            "occupied": occupied_count,
            "available": available_count,
            "summary": llm_summary
        })

    frame_num += 1

save_report_to_md(report_data)

# Release resources
cap.release()
cv2.destroyAllWindows()

# if window_closed:
#     print("Window closed properly and video capture terminated.")


End of video or can't retrieve frame.
